# CONVERT FILES WITH DUCK_DB

In [1]:
import duckdb
import os
import time

In [2]:
ZIP_PATH = r"..\data_processed\\guo_subs_europee\\"
ZIP_FOLDER = "guos_eu"

In [3]:
def time_it(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs) 
        end_time = time.time() 
        elapsed_time = end_time - start_time  
        print(f"Execution time: {elapsed_time:.4f} seconds")  
        return result
    return wrapper

## Ownership history

In [4]:
# path = "E:\dati_moody\ownership_history\links_2022\part-00000-8f9cac6d-cf88-4461-91b4-13c784cdf6a9-c000.snappy.parquet" 

In [5]:
eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE", "GR", 
    "HU", "IE", "IT", "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]


In [6]:

TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\*.parquet" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_raw\\key_financials\\key_financials_eur\\*.parquet"


def query(year, country, path):
    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            main.type_of_relation,
            main.shareholder_bvd_id,
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            {year} AS year,
            key_financials.number_of_employees AS sub_number_of_employees,
            key_financials.closing_date AS sub_closing_date,
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN (
            SELECT 
                number_of_employees, 
                closing_date,
                bvd_id_number,
                EXTRACT(YEAR FROM closing_date) AS financial_year  
            FROM 
                '{TEMP_TABLE_KEY_FINANCIALS}' 
        ) AS key_financials
        ON 
            main.guo_25 = key_financials.bvd_id_number
        AND 
            EXTRACT(YEAR FROM key_financials.closing_date) = {year} 
        WHERE 
            main."type_of_relation" IN ('GUO 25', 'ISH')
        AND 
            main."guo_25" LIKE '{country}%'
    """
    return query


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")

    query_ = query(year, country, path)
            
    conn = duckdb.connect()
    df = conn.execute(query_).fetchdf()
    return df


def convert_to_stata(df, output_path, country, year):
    os.makedirs(output_path, exist_ok=True)
    df.to_stata(f"{output_path}\\{country}_{year}.dta", write_index=False)


@time_it
def fetch_and_convert_to_stata(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    convert_to_stata(df, output_path, country, year)
    print(f"Data for {country} - {year} has been converted to Stata.")



### Test

In [7]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
# country = "IT"
# year = "2018"
# path = "..\\data_raw\\ownership_history\\links_2018\\*.parquet"
# output_path = r"..\data_processed\\guo_subs_europee\\subs_eu"


# df = get_ownership_data(
#     path=path,
#     year=year,
#     country=country,
# )
# df

# fetch_and_convert_to_stata(country=country, year=year, path=path, output_path=output_path)

In [ ]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\data_processed\\guo_subs_europee\\guos_eu"

total_start = time.time()
start_time = time.time()

for country in eu27_countries:
    for year in range(2007, 2022):
        year = str(year)
        try:
            start_time = time.time()
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue

total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors_subs.json", "w") as f:
    json.dump(errors, f)


In [ ]:
import shutil
ZIP_PATH = r"..\data_processed\\guo_subs_europee\\"

shutil.make_archive(ZIP_PATH, 'zip', ZIP_FOLDER)

In [ ]:
import json
with open("errors.json", "r") as f:
    errors = json.load(f)

errors

In [ ]:
dropbox_token = "j52rru17g3nj36g"

import dropbox

# Your Dropbox access token
access_token = 'your_dropbox_access_token'

# Path to the zip file you want to upload
zip_file_path = 'path_to_your_zip_file.zip'

# Destination path in Dropbox (e.g., "/folder_name/your_archive.zip")
dropbox_dest_path = '/your_dropbox_folder/your_archive_name.zip'

# Create a Dropbox client
dbx = dropbox.Dropbox(dropbox_token)

# Upload the file to Dropbox
with open(zip_file_path, 'rb') as f:
    dbx.files_upload(f.read(), dropbox_dest_path, mode=dropbox.files.WriteMode.overwrite)

print(f"File uploaded successfully to {dropbox_dest_path}")
